In [1]:
import os
import configparser

# Loading configurations
configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)

insert_amd_env_vars =  int(configParser.get('COMMON', 'insert_amd_env_vars'))
HSA_OVERRIDE_GFX_VERSION =  configParser.get('COMMON', 'HSA_OVERRIDE_GFX_VERSION')
ROCM_PATH =  configParser.get('COMMON', 'ROCM_PATH')

if(insert_amd_env_vars != 0):
    os.environ["HSA_OVERRIDE_GFX_VERSION"] = HSA_OVERRIDE_GFX_VERSION
    os.environ["ROCM_PATH"] = ROCM_PATH


In [2]:
from textwrap import wrap
import os
import gc
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

#TODO document jupyter
import pickle
import configparser
import sqlite3 as sl
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from pathlib import Path

configParser = configparser.RawConfigParser()   
configFilePath = r'configuration.txt'
configParser.read(configFilePath)
datasetPathDatabase =  configParser.get('COMMON', 'datasetPathDatabase') + '/dataset.db'
datasetPathDatabaseAdditional =  configParser.get('COMMON', 'datasetPathDatabase') + '/datasetAdditional.db'
datasetPathDatabaseVgg =  configParser.get('COMMON', 'datasetPathDatabase') + '/datasetFacesBlurred.db'
image_size =  int(configParser.get('COMMON', 'resizeImageTo'))
begin_with_image_size = int(configParser.get('COMMON', 'begin_with_image_size'))
unet1_dim =  int(configParser.get('COMMON', 'unet1_dim'))
unet2_dim =  int(configParser.get('COMMON', 'unet2_dim'))
unet1_image_size =  int(configParser.get('COMMON', 'unet1_image_size'))
audio_embs =  str(configParser.get('COMMON', 'audio_embs'))
audio_length_used =  configParser.get('evaluate_imagen', 'audio_length_used') 
model_filename =  configParser.get('evaluate_imagen', 'model_filename') + '_'  + audio_length_used +  's.pt'
cond_scale =  int(configParser.get('evaluate_imagen', 'cond_scale'))
datasetPathVideo =  configParser.get('COMMON', 'test_datasetPathVideo')
datasetPathGeneratedFaces =  configParser.get('evaluate_imagen', 'test_datasetPathGeneratedFaces')
generated_face_table_name =  configParser.get('evaluate_imagen', 'generated_face_table_name')


timesteps= int(configParser.get('COMMON', 'timesteps') )







#db_sampling_chunk = int(configParser.get('train_imagen', 'db_sampling_chunk'))
sampling_chunk = int(configParser.get('evaluate_imagen', 'sampling_chunk'))
preprocessing_chunk = int(configParser.get('evaluate_imagen', 'preprocessing_chunk'))



ignore_speaker_embedding = bool(int(configParser.get('evaluate_imagen', 'ignore_speaker_embedding') ))
ignore_speech_brain = bool(int(configParser.get('evaluate_imagen', 'ignore_speech_brain') ))
ignore_pyannote_titanet_speakernet = bool(int(configParser.get('evaluate_imagen', 'ignore_pyannote_titanet_speakernet') ))
ignore_audio_features = bool(int(configParser.get('evaluate_imagen', 'ignore_audio_features') ))
ignore_pyAudioAnalysis = bool(int(configParser.get('evaluate_imagen', 'ignore_pyAudioAnalysis') ))
ignore_librosa = bool(int(configParser.get('evaluate_imagen', 'ignore_librosa') ))
ignore_image_guide = bool(int(configParser.get('evaluate_imagen', 'ignore_image_guide') ))
ignore_additional_attributes = bool(int(configParser.get('evaluate_imagen', 'ignore_additional_attributes') ))
ignore_age = bool(int(configParser.get('evaluate_imagen', 'ignore_age') ))
ignore_gender = bool(int(configParser.get('evaluate_imagen', 'ignore_gender') ))
ignore_ethnicity = bool(int(configParser.get('evaluate_imagen', 'ignore_ethnicity') ))
ignore_language_spoken = bool(int(configParser.get('evaluate_imagen', 'ignore_language_spoken') ))


pd.set_option('display.max_columns', None)
pd.options.mode.chained_assignment = None



In [3]:
datasetPathDatabase =  configParser.get('COMMON', 'test_datasetPathDatabase') + '/dataset.db'

In [4]:
#import lancedb
#uri = datasetPathDatabase
#db = lancedb.connect(uri)

#table = db.open_table("video_stage1")
#videos = table.to_pandas()
#videos


In [5]:
def speaker_emb_preprocess(speaker_emb2):

    if(ignore_speaker_embedding or ignore_speech_brain):
        #print('ignore speech brain')
        return np.zeros((1,768))
    else:
        speaker_emb2 = pickle.loads(speaker_emb2)
        #print(speaker_emb2.print)
        speaker_emb2 = speaker_emb2.squeeze()
        speaker_emb2 = np.pad(speaker_emb2, (288), 'constant', constant_values=(0))
        speaker_emb2 = np.tile(speaker_emb2, (1, 1))
        speaker_emb2 = speaker_emb2 / 200.0
        #print(speaker_emb2.shape)
        #speaker_emb2 = np.array(speaker_emb2).tolist()
        #print(type(speaker_emb2))
        return speaker_emb2


def audio_emb_preprocess2(speaker_emb2):
    
    if(ignore_pyannote_titanet_speakernet or ignore_speaker_embedding):
        #print('ignore speech brain')
        return np.zeros((28,768))
    else:
        #print(speaker_emb2)
        speaker_emb2 = pickle.loads(speaker_emb2)
        a = speaker_emb2 #np.zeros(shape=(24, 512))
        a = a[-28:,:] # Actually only last 39 are the relevant ones
        #print('au')
        #print(a.shape)
        b = np.zeros(shape=(a.shape[0], 768-a.shape[1]))
        #c = np.zeros(shape=(28-a.shape[0], 768))
        arr = np.concatenate((a, b), axis=1)
        #arr = np.concatenate((arr, c), axis=0)
        arr = arr / (1 if(audio_embs == 'wav2vec') else 10 if(audio_embs == 'openl3')  else 1 if(audio_embs == 'pyannoteTitaNet') else 1)
        #print(str(arr.max()))
        #print(arr.shape)
        speaker_emb2 = np.array(arr)
        return speaker_emb2

def audio_features_preprocess(audio_features):
    #  79 belong to pyaudioanalysis
    # 111 belogn to liborsa
    if(ignore_audio_features):
        #print('ignore speech brain')
        return np.zeros((190,768))
    else:
        audio_features = pickle.loads(audio_features)
        #print(audio_features.shape) # 190 x 128
        audio_features = audio_features[0:190]
        
        #import sys
        #np.set_printoptions(threshold=sys.maxsize)
        #print(np.argwhere(audio_features == 0))

        if(ignore_pyAudioAnalysis and not ignore_librosa):
            zpa = np.zeros((190-110,128))
            audio_features = audio_features[80:190]
            audio_features = np.vstack((audio_features,zpa))
            #print('ignore pyAudioAnalysis')
            #print(audio_features.shape)
        elif(ignore_librosa and not ignore_pyAudioAnalysis):
            zpa = np.zeros((190-80,128))
            audio_features = audio_features[0:80]
            audio_features = np.vstack((audio_features,zpa))
            #print('ignore librosa')
            #print(audio_features.shape)
        else:
            audio_features = np.zeros((190,128))
            #print('ignore pyaudioanalysis and librosa')


        z1 = np.zeros((190,768-128))
        audio_features = np.hstack((audio_features,z1))
        audio_features = audio_features / 100.0
        #print(audio_features.shape)
    return audio_features

def audio_transformer_features_preprocess(video_id):
    #print(video_id)
    conAdditional = sl.connect(datasetPathDatabaseAdditional)
    cur = conAdditional.cursor()
    sql = '''SELECT AUDIO_FEATURES FROM AUDIO_TRANSFORMER WHERE VIDEO_ID = ?'''
    cur.execute(sql, [video_id])
    audio_features = cur.fetchall()
    #print(audio_features[0])
    audio_features = pickle.loads(audio_features[0][0])
    #print(audio_features.shape) # 514 x 768
    audio_features = audio_features.squeeze()
    audio_features = audio_features[0::3]#172*768
    #z1 = np.zeros((161,768-128))
    #audio_features = np.hstack((audio_features,z1))
    #audio_features = audio_features / 100.0
    return audio_features

import random
def image_guide_preprocess(face_id):
    #print(face_id)

    if(random.random() > 2):
        image_guide = np.zeros((49,768))
        #print(image_guide.shape)
    else:
        conAdditional = sl.connect(datasetPathDatabaseVgg)
        cur = conAdditional.cursor()
        sql = '''SELECT BLURRED_FACE_EMB FROM FACES_BLURRED WHERE FACE_ID = ?'''
        cur.execute(sql, [face_id])
        image_guide = cur.fetchall()
        #print(image_guide)
        image_guide = pickle.loads(image_guide[0][0])
        image_guide = image_guide.squeeze() #197 x 768
        image_guide = image_guide[1::4] #49 x 768
    return image_guide


#boxBlurMin =  int(configParser.get('extractVggBlurred', 'boxBlurMin'))
#boxBlurMax =  int(configParser.get('extractVggBlurred', 'boxBlurMax'))

#gaussianBlurMin =  int(configParser.get('extractVggBlurred', 'gaussianBlurMin'))
#gaussianBlurMax =  int(configParser.get('extractVggBlurred', 'gaussianBlurMax'))


from PIL import Image,ImageFilter
import random
#import cv2
import numpy as np
#from matplotlib import pyplot as plt
  


def image_guide_preprocess_low_res_dummy(path):
    #print(face_id)

    image_guide = np.zeros((1, 768))
    return image_guide

def image_guide_preprocess_low_res(path):
    #print(face_id)


    if(random.random() > 2): #never
        image_guide = np.zeros((49, 768))
        #print(image_guide.shape)
    else:

        image = Image.open(path).convert('RGB')

        #print(image.size)

        w_s = image_size / (1+2 * 0.4)
        h_s = image_size / (1+2 * 0.4)

        image = image.crop((0.2*w_s, 0.0*h_s, 1.6*w_s, 1.4*h_s))

        image = image.resize((image_size,image_size))

        image = image.resize((begin_with_image_size,begin_with_image_size))
        im = image
        

        #print('saving')
        #image.save('opop.png')

        #print(np.array(image,np.float32).shape)

        pix = np.array(image, np.float32)
        pix = np.moveaxis(pix, -1, 0)

        pix = pix / 255
        image.close()
        im.close()
        return pix
    return image_guide




In [6]:



def process_age(age):
    if(ignore_age or ignore_additional_attributes):
        x = np.zeros(768)
        x[767] = 1
        return x
    else:
        x = np.ones(768) * (age / 100.0)
        x[767] = 0
        return x

def process_gender(gender):
    if(ignore_gender or ignore_additional_attributes):
        return np.zeros(768)
    elif(gender == 'man'):
        return np.ones(768)
    elif(gender == "woman"):
        return np.ones(768) * -1
    else:
        return np.zeros(768)
    
# TODO
def process_ethnicity(eth):
    x = np.zeros(768)
    if(ignore_ethnicity or ignore_additional_attributes):
        x = x
    elif(eth == "indian"):
        x[0] = 1
    elif(eth == "asian"):
        x[16]=1
    elif(eth == "latino hispanic"):
        x[2]=1
    elif(eth == "black"):
        x[3]=1
    elif(eth == "middle eastern"):
        x[4]=1
    elif(eth == "white"):
        x[5]=1 
    else:
        x = x
    return x     

def process_language(lan):
    x = np.zeros(768)
    if(ignore_language_spoken or ignore_additional_attributes):
        x = x
    elif(lan == "Arabic"):
        x[0] = 1
    elif(lan == "Portuguese"):
        x[16]=1
    elif(lan == "Romansh_Sursilvan"):
        x[2]=1
    elif(lan == "Japanese"):
        x[3]=1
    elif(lan == "Ukranian"):
        x[4]=1
    elif(lan == "German"):
        x[5]=1   
    elif(lan == "Chinese_China"):
        x[6]=1   
    elif(lan == "Welsh"):
        x[7]=1  
    elif(lan == "English"):
        x[8]=1
    elif(lan == "Kabyle"):
        x[9]=1 
    elif(lan == "Kyrgyz"):
        x[10]=1
    elif(lan == "Georgian"):
        x[11]=1
    elif(lan == "Persian"):
        x[12]=1 
    elif(lan == "French"):
        x[13]=1
    elif(lan == "Interlingua"):
        x[14]=1
    elif(lan == "Swedish"):
        x[15]=1
    elif(lan == "Spanish"):
        x[16]=1 
    elif(lan == "Dhivehi"):
        x[17]=1
    elif(lan == "Kinyarwanda"):
        x[18]=1 
    elif(lan == "Tatar"):
        x[19]=1
    elif(lan == "Hakha_Chin"):
        x[20]=1 
    elif(lan == "Tamil"):
        x[21]=1 
    elif(lan == "Greek"):
        x[22]=1
    elif(lan == "Latvian"):
        x[23]=1 
    elif(lan == "Russian"):
        x[24]=1
    elif(lan == "Breton"):
        x[25]=1
    elif(lan == "Catalan"):
        x[26]=1    
    elif(lan == "Maltese"):
        x[27]=1 
    elif(lan == "Slovenian"):
        x[28]=1    
    elif(lan == "Indonesian"):
        x[29]=1    
    elif(lan == "Dutch"):
        x[30]=1
    elif(lan == "Chinese_Taiwan"):
        x[31]=1 
    elif(lan == "Sakha"):
        x[32]=1 
    elif(lan == "Polish"):
        x[33]=1 
    elif(lan == "Czech"):
        x[34]=1 
    elif(lan == "Romanian"):
        x[35]=1 
    elif(lan == "Mangolian"):
        x[36]=1 
    elif(lan == "Italian"):
        x[37]=1 
    elif(lan == "Chinese_Hongkong"):
        x[38]=1 
    elif(lan == "Estonian"):
        x[39]=1 
    elif(lan == "Basque"):
        x[40]=1 
    elif(lan == "Esperanto"):
        x[41]=1 
    elif(lan == "Frisian"):
        x[42]=1 
    elif(lan == "Turkish"):
        x[43]=1 
    elif(lan == "Chuvash"):
        x[44]=1 
    else:
        x = x
    return x       

In [7]:

def process_input(row):
    #print(row)
    age = row["caption_a"]
    ethnicity=row["caption_e"]
    gender=row["caption_g"]
    language=row["caption_l"]
    speaker_emb=row["SPEAKER_EMB"]
    audio_emb=row["AUDIO_EMB"]
    audio_features=row["AUDIO_FEATURES"]
    image_guide=row["image_guide"]
    #print(image_guide)
    #print(speaker_emb)
    speaker_emb = np.asarray(speaker_emb, dtype=np.float32)
    speaker_emb = speaker_emb.squeeze()
    audio_emb = np.asarray(audio_emb, dtype=np.float32)
    audio_emb = audio_emb.squeeze()
    #print(speaker_emb.shape)
    h = np.vstack((age, ethnicity))
    h = np.vstack((h, gender))
    h = np.vstack((h, language))
    h = np.vstack((h, speaker_emb))
    h = np.vstack((h, audio_emb))
    h = np.vstack((h,audio_features))
    h = np.vstack((h,image_guide))
    #print('aaaaaaaaaaa')
    #print(h.shape)
    j = np.zeros(768)
    j = np.tile(j,(256-h.shape[0],1))
    h = np.vstack((h, j))
    #print(h.shape)
    
    return h





def process_image_path(path):
    
    image = Image.open(path).convert('RGB')

    w_s = image_size / (1+2 * 0.4)
    h_s = image_size / (1+2 * 0.4)

    #print(image.size)
    image = image.crop((0.2*w_s,0.0*h_s,1.6*w_s,1.4*h_s))
    image = image.resize((image_size,image_size))

    #print('saving')
    #image.save(str(random.random()) + '.png')

    

    #print(np.array(image,np.float32).shape)
    pix = np.array(image,np.float32)
    pix = np.moveaxis(pix, -1, 0)
    
    pix = pix / 255
    image.close()
    return pix

In [8]:

import pathlib
import lancedb
uri = datasetPathDatabase
db = lancedb.connect(uri)

try:
    db.drop_table(generated_face_table_name)
except:
    pass



def save_generated_image(row):
    id = row['videos_id']
    row_videos = videos_df.loc[videos_df['id'] == id]
    generated_image = row['generated']
    
    absPathVideo = row_videos['video_path'].iloc[0]


    absPathGeneratedFace = absPathVideo.replace(datasetPathVideo,datasetPathGeneratedFaces)
    absPathGeneratedFace = os.path.splitext(absPathGeneratedFace)[0]
    absPathGeneratedFace = absPathGeneratedFace + "_generated_face_"+ ".png"
    pathlib.Path(os.path.dirname(absPathGeneratedFace)).mkdir(parents=True, exist_ok=True) 

    #print(row)
    #try:
    #    print(generated_image)
    #except:
    #    pass

    #try:
    #    print(generated_image.shape)
    #except:
    #    pass

    #print(generated_image)
    generated_image.save(absPathGeneratedFace)

    p_small = pathlib.Path(os.path.dirname(absPathVideo))
    p_big = p_small.parent.absolute()
    df_table2 = pd.DataFrame()
    df_table2 = df_table2.append({'id':int(id),'id_true': int(id),
                    'generated_face_path': absPathGeneratedFace,'user':p_big.name}, ignore_index=True)
    
    
    try:
        table_stage2 = db.open_table(generated_face_table_name)
        table_stage2.add(df_table2)
    except:
        db.create_table(generated_face_table_name, df_table2)

    return




In [9]:

import dask.dataframe as dd

db = lancedb.connect(uri)

table = db.open_table("video_stage1")
videos = table.to_pandas()
videos_df = videos
videos = dd.from_pandas(videos,chunksize=preprocessing_chunk)
videos

,id,video_path,face_path,user,blurred_face_path,features_path,age,gender,ethnicity,vector,stage
npartitions=3,,,,,,,,,,,
0,int64,object,object,object,object,object,float64,object,object,object,int64
4,...,...,...,...,...,...,...,...,...,...,...
8,...,...,...,...,...,...,...,...,...,...,...
10,...,...,...,...,...,...,...,...,...,...,...


In [10]:
import multiprocessing
import train_imagen_batch
import dask.dataframe as dd
import math

j = 0

no_of_video_chunks = math.ceil(len(videos)/preprocessing_chunk)
print("No. of preprocessing chunks: " + str(no_of_video_chunks))

while(True):

    videos_chunk = videos.partitions[j:j+1]
    print(len(videos_chunk))

    features = pd.DataFrame()
    print("Getting Data")
    for index, row in videos_chunk.iterrows():

        with open(row['features_path'], 'rb') as handle:
            feature = pickle.load(handle)
            feature['videos_id'] = row['id']
            #print(feature)
            features = features.append(feature)

    #print(features)







    #print(features)
    data_frame = features[["ID","FACE_ID","image_path","caption_a","caption_e","caption_g","caption_l","videos_id"]]
    data_frame['SPEAKER_EMB'] = features['SPEAKER_EMB']
    data_frame['AUDIO_EMB'] = features['AUDIO_EMB']
    data_frame['image_guide'] = features['image_guide']
    data_frame['AUDIO_FEATURES'] = features['AUDIO_FEATURES']


    data_frame['SPEAKER_EMB'] = data_frame['SPEAKER_EMB'].apply(lambda x: speaker_emb_preprocess(x))
    data_frame['AUDIO_EMB'] = data_frame['AUDIO_EMB'].apply(lambda x: audio_emb_preprocess2(x))
    data_frame['AUDIO_FEATURES'] = data_frame['AUDIO_FEATURES'].apply(lambda x: audio_features_preprocess(x))
    data_frame = data_frame.drop(['ID'], axis=1)
    data_frame['image_guide'] = data_frame['image_path'].apply(lambda x: image_guide_preprocess_low_res_dummy(x))
    data_frame = data_frame.drop(['FACE_ID'], axis=1)
    data_frame['caption_a'] = data_frame['caption_a'].apply(lambda x: process_age(x))
    data_frame['caption_g'] = data_frame['caption_g'].apply(lambda x: process_gender(x))
    data_frame['caption_l'] = data_frame['caption_l'].apply(lambda x: process_language(x))
    data_frame['caption_e'] = data_frame['caption_e'].apply(lambda x: process_ethnicity(x))
    data_frame['low_res_image'] = data_frame['image_path'].apply(lambda x: image_guide_preprocess_low_res(x))

    data_frame['INPUT'] = data_frame['SPEAKER_EMB']

    data_frame['INPUT'] = data_frame.apply(process_input,args=(),axis=1)


    data_frame = data_frame.drop(['caption_e', 'caption_g','caption_l','SPEAKER_EMB','AUDIO_EMB'], axis=1)


    data_frame['image_path'] = data_frame['image_path'].apply(lambda x: process_image_path(x))

    print("Data Preprocessed...")






    part = data_frame[['INPUT','low_res_image']]



    input0 = np.array(part['INPUT'])
    input0 = np.array([np.array(xi) for xi in input0])
    input0[np.isnan(input0)] = 0
    input0[input0 > 10] = 10
    input0[input0 < -10] = -10
    #print(sys.getsizeof(input))


    input2 = np.array(part['low_res_image'])
    input2 = np.array([np.array(xi) for xi in input2])
    input2.squeeze()
    #print(sys.getsizeof(input2))





    proc = multiprocessing.Process(target=train_imagen_batch.sample_batch_unet1, args=(input0,input2,model_filename,image_size,unet1_dim,unet2_dim,timesteps,begin_with_image_size,unet1_image_size,cond_scale,sampling_chunk,))
    proc.start()
    proc.join()

    with open('evaluate_imagen2.tmp', 'rb') as handle:
        images = pickle.load(handle)

    print("images")
    print(images)


    data_frame.reset_index(drop=True, inplace=True)
    data_frame['generated'] = pd.DataFrame(data=images)
    data_frame.apply(save_generated_image,args=(),axis=1)





    del images
    del features
    del data_frame
    del input0
    del input2
    gc.collect()

    j = j+1
    print("no of chunks:" + str(no_of_video_chunks))
    print(j)
    if(j == no_of_video_chunks):
        break
        




No. of preprocessing chunks: 3
4
Getting Data
Data Preprocessed...


/tmp/ipykernel_180262/2406898082.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(feature)
/tmp/ipykernel_180262/2406898082.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(feature)
/tmp/ipykernel_180262/2406898082.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(feature)
/tmp/ipykernel_180262/2406898082.py:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  features = features.append(feature)
/home/gamal/anaconda3/envs/ds2f_m_i/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See

The base dimension of your u-net should ideally be no smaller than 128, as recommended by a professional DDPM trainer https://nonint.com/2022/05/04/friends-dont-let-friends-train-small-diffusion-models/
Using model file imagen_features_low_res_asis_24s.pt
checkpoint loaded from imagen_features_low_res_asis_24s.pt


sampling loop time step: 100%|██████████| 256/256 [00:22<00:00, 11.36it/s]
2it [00:41, 20.61s/it]
sampling loop time step: 100%|██████████| 256/256 [00:22<00:00, 11.48it/s]
2it [00:40, 20.30s/it]


images
[<PIL.Image.Image image mode=RGB size=32x32 at 0x7FE08DCD0DC0>, <PIL.Image.Image image mode=RGB size=32x32 at 0x7FE08DCD0F70>, <PIL.Image.Image image mode=RGB size=32x32 at 0x7FE08DCD0D00>, <PIL.Image.Image image mode=RGB size=32x32 at 0x7FE08DCD0E20>]
no of chunks:3
1
4
Getting Data


...

/tmp/ipykernel_180262/3921310896.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_table2 = df_table2.append({'id':int(id),'id_true': int(id),
/tmp/ipykernel_180262/3921310896.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_table2 = df_table2.append({'id':int(id),'id_true': int(id),
/tmp/ipykernel_180262/3921310896.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_table2 = df_table2.append({'id':int(id),'id_true': int(id),
/tmp/ipykernel_180262/3921310896.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_table2 = df_table2.append({'id':int(id),'id_true': int(id),
/tmp/ipykernel_180262/2406898082.py:24: FutureWa

The base dimension of your u-net should ideally be no smaller than 128, as recommended by a professional DDPM trainer https://nonint.com/2022/05/04/friends-dont-let-friends-train-small-diffusion-models/
Using model file imagen_features_low_res_asis_24s.pt
checkpoint loaded from imagen_features_low_res_asis_24s.pt


sampling loop time step: 100%|██████████| 256/256 [00:22<00:00, 11.31it/s]
2it [00:41, 20.91s/it]
sampling loop time step: 100%|██████████| 256/256 [00:20<00:00, 12.53it/s]
2it [00:39, 19.71s/it]


images
[<PIL.Image.Image image mode=RGB size=32x32 at 0x7FE08DCDCD90>, <PIL.Image.Image image mode=RGB size=32x32 at 0x7FE08DCDCE50>, <PIL.Image.Image image mode=RGB size=32x32 at 0x7FE08DCDC9A0>]
no of chunks:3
3


/tmp/ipykernel_180262/3921310896.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_table2 = df_table2.append({'id':int(id),'id_true': int(id),
/tmp/ipykernel_180262/3921310896.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_table2 = df_table2.append({'id':int(id),'id_true': int(id),
/tmp/ipykernel_180262/3921310896.py:43: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_table2 = df_table2.append({'id':int(id),'id_true': int(id),
